# Εργαστήριο 4
Όπως πάντα, ξεκινήστε **εκτελώντας τα επόμενα δύο κελιά**

*(Αυτό θα πρέπει να γίνεται επίσης κάθε φορά που κάνετε kernel restart)*

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

## Αυτόματη προσαρμογή στο εύρος των σημάτων
Ένα μεγάλο πλεονέκτημα της Chisel είναι ότι η **ίδια έκφραση** μπορεί να κατασκευάσει πολλαπλά κυκλώματα, ανάλογα με το **εύρος** (width) των σημάτων που μετέχουν στην έκφραση. Για παράδειγμα, η έκφραση

~~~scala
io.y := io.a & io.b
~~~

θα φτιάξει το κύκλωμα `y = a AND b` (μία πύλη AND) αν τα a και b έχουν εύρος 1 bit.

Εάν όμως το εύρος των a και b είναι π.χ. 4 bits, **η ίδια έκφραση** (`io.y := io.a & io.b`) θα φτιάξει τα εξής (4 πύλες AND):
~~~
y(3) = a(3) AND b(3)
y(2) = a(2) AND b(2)
y(1) = a(1) AND b(1)
y(0) = a(0) AND b(0)
~~~

Το εύρος του αποτελέσματος των τελεστών πράξεων με bits προσαρμόζεται ως εξής:

| τελεστής | εύρος αποτελέσματος |
| --- | --- |
| ~x (NOT) | εύρος(x) |
| x & y (AND) | max(εύρος(x),εύρος(y)) |
| x \| y (OR) | max(εύρος(x),εύρος(y)) |
| x ^ y (XOR) | max(εύρος(x),εύρος(y)) |

Αυτή την ιδιότητα προσαρμογής μπορούμε να την εκμεταλλευτούμε για την περιγραφή **παραμετρικών κυκλωμάτων** (μεταβλητού εύρους σημάτων εισόδου-εξόδου, όπως θα φανεί στη συνέχεια.

## Περιγραφή παραμετρικών κυκλωμάτων
**Μελετήστε** το επόμενο κύκλωμα, στο οποίο ορίζεται παραμετρικά (με τη μεταβλητή `n`) το εύρος των σημάτων εισόδου-εξόδου.

Όταν χρησιμοποιηθεί το κύκλωμα αυτό, το ακριβές εύρος ορίζεται κατά τη δημιουργία του, π.χ. το επόμενο δημιουργεί ένα κύκλωμα με εισόδους-εξόδους εύρους 8 bits:
~~~scala
new SomeLogic(8)
~~~

In [ ]:
class SomeLogic(n: Int) extends Module {
  val io = IO(new Bundle {
    val a = Input(UInt(n.W))
    val b = Input(UInt(n.W))
    val y = Output(UInt(n.W))
  })
  
  io.y := io.a & io.b
}

Μπορείτε να **ελέγξετε** τη λειτουργία του κυκλώματος για συγκεκριμένο εύρος (στο επόμενο παράδειγμα 8 bits):

In [ ]:
test(new SomeLogic(8)) { c =>
  c.io.a.poke("b00101101".U)
  c.io.b.poke("b10110111".U)
  c.io.y.expect("b00100101".U)
}
println("SUCCESS!!")

## Πρόσθεση και αφαίρεση
Όπως έχει αναφερθεί στο προηγούμενο εργαστήριο, η Chisel μας παρέχει τους τελεστές `+` και `-` για να περιγράψουμε κυκλώματα που εκτελούν **πρόσθεση** και **αφαίρεση** αντίστοιχα. Και αυτοί οι τελεστές προσαρμόζουν αυτόματα το εύρος της εξόδου σύμφωνα με το εύρος των σημάτων εισόδου:

| τελεστής | εύρος αποτελέσματος |
| --- | --- |
| x + y (πρόσθεση) | max(εύρος(x),εύρος(y)) |
| x - y (αφαίρεση) | max(εύρος(x),εύρος(y)) |

**Μελετήστε** το επόμενο παραμετρικό κύκλωμα που αθροίζει δύο αριθμούς των `n` bits. Το αποτέλεσμα είναι πάλι `n` bits.

In [ ]:
class AdderTest(n: Int) extends Module {
  val io = IO(new Bundle {
    val a = Input(UInt(n.W))
    val b = Input(UInt(n.W))
    val y = Output(UInt(n.W))
  })
  
  io.y := io.a + io.b
}

**Ελέγξτε** μια υλοποίηση με αριθμούς 8 bits:

In [ ]:
test(new AdderTest(8)) { c =>
  c.io.a.poke("b00101101".U)
  c.io.b.poke("b10110111".U)
  c.io.y.expect("b11100100".U)

  c.io.a.poke("b11111111".U)
  c.io.b.poke("b00000001".U)
  c.io.y.expect("b00000000".U) // no carry-out capture!
}
println("SUCCESS!!")

## Λήψη τελικού κρατουμένου εξόδου
Στον προηγούμενο αθροιστή δεν έχουμε δυνατότητα λήψης του τελικού κρατουμένου εξόδου (αν παράγεται). Η Chisel όμως παρέχει παραλλαγές των τελεστών πρόσθεσης και αφαίρεσης, οι οποίοι επεκτείνουν το αποτέλεσμα **σε ένα επιπλέον bit** στα αριστερά. Αυτό το bit θα περιέχει το τελικό κρατούμενο εξόδου, αν αυτό υπάρχει.

| τελεστής | εύρος αποτελέσματος |
| --- | --- |
| x +& y (πρόσθεση με επέκταση) | max(εύρος(x),εύρος(y))+1 |
| x -& y (αφαίρεση με επέκταση) | max(εύρος(x),εύρος(y))+1 |

**Μελετήστε** στο επόμενο κύκλωμα πώς μπορύμε με τη χρήση του τελεστή `+&` να λάβουμε το αποτέλεσμα `y` (bits n-1 έως 0 της μεταβλητής `sum`) και το κρατούμενο εξόδου `c` (bit n της μεταβλητής `sum`) της πρόσθεσης δύο αριθμών (a και b) εύρους `n` bits: 

In [ ]:
class AdderTest2(n: Int) extends Module {
  val io = IO(new Bundle {
    val a = Input(UInt(n.W))
    val b = Input(UInt(n.W))
    val y = Output(UInt(n.W))
    val c = Output(UInt(1.W))
  })
  
  val sum = io.a +& io.b
  io.y := sum(n-1,0)
  io.c := sum(n)
}

**Ελέγξτε** μια υλοποίηση με αριθμούς 8 bits:

In [ ]:
test(new AdderTest2(8)) { c =>
  c.io.a.poke("b00101101".U)
  c.io.b.poke("b10110111".U)
  c.io.y.expect("b11100100".U)
  c.io.c.expect(0.U)

  c.io.a.poke("b11111111".U)
  c.io.b.poke("b00000001".U)
  c.io.y.expect("b00000000".U)
  c.io.c.expect(1.U) // carry-out
}
println("SUCCESS!!")

## Άσκηση 1: Επιλογή πρόσθεσης ή αφαίρεσης
Το ζητούμενο είναι να περιγράψετε το module `AddSub`, το οποίο δέχεται τις εισόδους `a` `b` με παραμετρικό εύρος `n` και παράγει την έξοδο `out` με εύρος επίσης `n`. Το module εκτελεί εναλλακτικά πρόσθεση ή αφαίρεση (**χωρίς τελικό κρατούμενο**), ανάλογα με την τιμή τρίτου σήματος εισόδου `sub` εύρους 1 bit (0 = πρόσθεση, 1 = αφαίρεση), χρησιμοποιώντας τη δομή `when`.

**Συμπληρώστε** στο επόμενο κελί.

In [ ]:
class AddSub(n: Int) extends Module {
  val io = IO(new Bundle {
    val a = // ..συμπληρώστε..
    val b = // ..συμπληρώστε..
    val sub = // ..συμπληρώστε..
    val out = // ..συμπληρώστε..
  })
  
  // ..συμπληρώστε..    
    
}

Δοκιμάστε το module που φτιάξατε με εύρος εισόδων/εξόδου = 8 bits. Οι δυαδικές τιμές ελέγχου και το αντίστοιχο αποτέλεσμα θα πρέπει να είναι:

~~~
00101101 + 10110111 = 11100100
11111111 + 00000001 = 00000000

00101101 - 10110111 = 01110110
11111111 - 00000001 = 11111110
~~~

Θυμηθείτε ότι οι δυαδικές σταθερές UInt γράφονται στην Chisel ως π.χ. `"b10110111"`.

**Συμπληρώστε** τα κατάλληλα poke() και expect() στο επόμενο κελί:

In [ ]:
test(new AddSub(8)) { c =>
  c.io.sub.poke(0.U)  // test addition
    
  c.io.a.poke("b00101101".U)
  c.io.b.poke("b10110111".U)
  c.io.out.expect(..συμπληρώστε..)

  c.io.a.poke("b11111111".U)
  c.io.b.poke("b00000001".U)
  c.io.out.expect(..συμπληρώστε..) 

  // ..συμπληρώστε όμοια τα 2 παραδείγματα για την αφαίρεση..

}
println("SUCCESS!!")

## Άσκηση 2: Αριθμητική λογική μονάδα με παραμετρικό εύρος δεδομένων
Με βάση τα προηγούμενα (και το [προηγούμενο εργαστήριο](https://nbviewer.org/urls/mixstef.github.io/courses/comparch/lab3.ipynb)), κατασκευάστε (**συμπληρώστε στο επόμενο κελί**) παραμετρικό κύκλωμα `Alu` (αριθμητική-λογική μονάδα), το οποίο θα έχει τις εξής εισόδους-εξόδους:

* `a`, `b` (είσοδοι): οι δύο αριθμοί εισόδου, `n` bits εύρος ο καθένας
* `sel` (είσοδος): σήμα επιλογής πράξης, 2 bits εύρος
* `sub` (είσοδος): σήμα επιλογής πρόσθεσης ή αφαίρεσης, 1 bit εύρος
* `out` (έξοδος): αποτέλεσμα πράξης, `n` bits εύρος

**Η λειτουργία του κυκλώματος πρέπει να είναι η εξής:**

| sel(1) | sel(0) | out |
| :---: | :---: | :---: |
| 0 | 0 | a AND b |
| 0 | 1 | a OR b |
| 1 | 0 | a XOR b |
| 1 | 1 | a + b (αν sub=0), a - b (αν sub=1) |

**Υπόδειξη:** Μπορείτε να γράψετε nested δομές `when` μέσα σε άλλα `when`. Μπορείτε επίσης να συνδυάσετε συνθήκες π.χ. (προσέξτε πώς μπαίνουν οι παρενθέσεις):
~~~scala
when (io.x===0.U && io.y===1.U)  {
~~~


In [ ]:
class Alu(n: Int) extends Module {
  val io = IO(new Bundle {
    val a = // ..συμπληρώστε..
    val b = // ..συμπληρώστε..
    val sel = // ..συμπληρώστε..
    val sub = // ..συμπληρώστε..
    val out = // ..συμπληρώστε..
  })
  
  // ..συμπληρώστε..
    
}

### Ελέγξτε την υλοποίησή σας
Βεβαιωθείτε ότι το κύκλωμά σας περνάει επιτυχώς τον έλεγχο **εκτελώντας το επόμενο κελί**.

In [ ]:
import scala.util.{Random => rnd }
test(new Alu(8)) { c =>
  for (i <- 0 until 10) {     // 10 φορές
    val a = rnd.nextInt(256)  // για τυχαίους αριθμούς a
    val b = rnd.nextInt(256)  // και b
    c.io.a.poke(a.U)
    c.io.b.poke(b.U)

    // test AND
    c.io.sel.poke("b00".U)
    c.io.out.expect((a & b).U)
    // test OR
    c.io.sel.poke("b01".U)
    c.io.out.expect((a | b).U)
    // test XOR
    c.io.sel.poke("b10".U)
    c.io.out.expect((a ^ b).U)
    // test ADD
    val total = a+b
    val sum = total & 0xFF  // απομάκρυνση κρατουμένου εξόδου, αν υπάρχει
    c.io.sel.poke("b11".U)  
    c.io.sub.poke(0.U)
    c.io.out.expect(sum.U)
    // test SUB
    val total2 = a-b
    val sum2 = total2 & 0xFF  // απομάκρυνση κρατουμένου εξόδου, αν υπάρχει
    c.io.sel.poke("b11".U) 
    c.io.sub.poke(1.U)
    c.io.out.expect(sum2.U)

  }
}
println("SUCCESS!!")

### Ανεβάστε τη λύση σας στο opencourses
Ανεβάστε το τελικό σας notebook όπου φαίνονται και τα αποτελέσματα της εκτέλεσης στο opencourses (**Εργασία 1**) έως και την **Δευτέρα 30/10**.

*(Ίσως χρειαστεί να κάνετε zip το αρχείο ipynb για να γίνει αποδεκτό από το σύστημα)*
